In [179]:
import json
import gurobipy as gp
from gurobipy import GRB

with open('parametros_preasignacion.json', encoding='utf8') as f:
    data = json.load(f)

cantidad_de_horas_por_ramo_por_curso = data['cantidad_de_horas_por_ramo_por_curso']
ramos_y_cursos_dictados_por_profesor = data['ramos_y_cursos_dictados_por_profesor']
disponibilidad_horaria = data['disponibilidad_horaria']
clases_ya_asignadas = data['clases_ya_asignadas']
preferencias_profesores = data['preferencias_profesores']

# Se necesita saber a qué cursos PUEDE (nada de querer todavía) hacer clases cada profesor, dada las asignaciones previas que ya tiene. 

In [180]:
# ! Formar tuplas profesores-ramos-cursos

def tuplas_ramos_cursos(cantidad_de_horas_por_ramo_por_curso):
    lista_tuplas = []
    for c in cantidad_de_horas_por_ramo_por_curso:
        for r in cantidad_de_horas_por_ramo_por_curso[c]:
            lista_tuplas.append((r,c))
    return lista_tuplas

def tuplas_profesores_ramos_cursos(cantidad_de_horas_por_ramo_por_curso):
    lista_trios = []
    for p in ramos_y_cursos_dictados_por_profesor:
        for r,c in ramos_y_cursos_dictados_por_profesor[p]:
            lista_trios.append((p,r,c))
    return lista_trios

In [181]:
model = gp.Model("Preasignación de cursos a profesores")

cursos = list(cantidad_de_horas_por_ramo_por_curso.keys())
profesores = list(ramos_y_cursos_dictados_por_profesor.keys())
# ramos = ['Matemáticas', 'Lenguaje', 'Historia', 'Alemán']

duplas_rc = tuplas_ramos_cursos(cantidad_de_horas_por_ramo_por_curso)
trios_prc = tuplas_profesores_ramos_cursos(cantidad_de_horas_por_ramo_por_curso)

# ! Otros conjuntos importantes

y = model.addVars(trios_prc, vtype=GRB.BINARY, name="y")
w = model.addVars(duplas_rc, vtype=GRB.BINARY, name = "w")

In [182]:
model.addConstrs((sum(y[p,r,c] for p,s,d in trios_prc if r == s if c == d) + w[r,c] == 1 for r,c in duplas_rc),name="R1");

In [183]:
model.addConstrs((sum(y[p,r,c] * cantidad_de_horas_por_ramo_por_curso[c][r] for p,r,c in trios_prc if p == q) <= disponibilidad_horaria[q] for q in ramos_y_cursos_dictados_por_profesor),name="R2");

In [184]:
model.addConstrs((y[p,r,c] == 1 for r,p,c in clases_ya_asignadas),name="R3");

In [185]:
obj = 50 * sum(w[r,c] * cantidad_de_horas_por_ramo_por_curso[c][r] for c in cantidad_de_horas_por_ramo_por_curso for r in cantidad_de_horas_por_ramo_por_curso[c]) # ? Minimizar la cantidad de cursos asignados que hay que parchar
model.setObjective(obj, GRB.MINIMIZE)


model.update()
model.write('model.lp')
# model.computeIIS()
# removed =[]
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s' % c.constrName)
#         # Remove a single constraint from the model
#         removed.append(str(c.constrName))
#         model.remove(c)

model.optimize()
model.write('out_preasignacion.sol')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 111 rows, 144 columns and 185 nonzeros
Model fingerprint: 0x30ceb041
Variable types: 0 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [5e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 15800.000000
Presolve removed 101 rows and 128 columns
Presolve time: 0.00s
Presolved: 10 rows, 16 columns, 32 nonzeros
Found heuristic solution: objective 9400.0000000
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      9400.00000 9400.00000  0.00%     -    0s

Explor

In [186]:
with open ('out_preasignacion.sol') as f:
    profesores_faltantes = []
    ocupaciones_profesores = []
    for line in f.readlines()[2:]:
        if line[-2] == '1' and line[0] == 'w':
            line = line.strip('\n')
            line = line[2:-3]
            line = line.split(',')
            profesores_faltantes.append(line)
        if line[-2] == '1' and line[0] == 'y':
            line = line.strip('\n')
            line = line[2:-3]
            line = line.split(',')
            ocupaciones_profesores.append(line)

suma = 0
for elem in profesores_faltantes:
    suma += cantidad_de_horas_por_ramo_por_curso[elem[1]][elem[0]]
    print(f'Al curso {elem[1]} no fue posible asignarle profesor de {elem[0]} ({cantidad_de_horas_por_ramo_por_curso[elem[1]][elem[0]]} horas)')
print(f'El total de horas que no se pudieron asignar fue de {suma} horas')
horas_placeholder = 1.5 * suma # ! Esta es una estimación que considera la cantidad de horas placeholder 
                               # ! que faltarían y a la vez deja espacio para las prioridades


suma = 0
for i in range(len(ocupaciones_profesores)):
    if i+1 != len(ocupaciones_profesores):
        if ocupaciones_profesores[i][0] == ocupaciones_profesores[i+1][0]:
            suma += cantidad_de_horas_por_ramo_por_curso[ocupaciones_profesores[i][2]][ocupaciones_profesores[i][1]]
        if ocupaciones_profesores[i][0] != ocupaciones_profesores[i+1][0]:
            suma += cantidad_de_horas_por_ramo_por_curso[ocupaciones_profesores[i][2]][ocupaciones_profesores[i][1]]
            print(f'Profesor {ocupaciones_profesores[i][0]} tiene {suma} horas asignadas')
            suma = 0
    if i+1 == len(ocupaciones_profesores):
        suma += cantidad_de_horas_por_ramo_por_curso[ocupaciones_profesores[i][2]][ocupaciones_profesores[i][1]]
        print(f'Profesor {ocupaciones_profesores[i][0]} tiene {suma} horas asignadas')


Al curso 4A no fue posible asignarle profesor de Orientación (1 horas)
Al curso 4A no fue posible asignarle profesor de Educación_Física (3 horas)
Al curso 4A no fue posible asignarle profesor de Ciencias_Naturales (3 horas)
Al curso 4A no fue posible asignarle profesor de Religión (2 horas)
Al curso 4A no fue posible asignarle profesor de Taller_Comprensión_Lectora (1 horas)
Al curso 4A no fue posible asignarle profesor de Música (2 horas)
Al curso 4A no fue posible asignarle profesor de Tecnología (2 horas)
Al curso 4A no fue posible asignarle profesor de Artes_Visuales (2 horas)
Al curso 4B no fue posible asignarle profesor de Orientación (1 horas)
Al curso 4B no fue posible asignarle profesor de Educación_Física (3 horas)
Al curso 4B no fue posible asignarle profesor de Lenguaje (7 horas)
Al curso 4B no fue posible asignarle profesor de Ciencias_Naturales (3 horas)
Al curso 4B no fue posible asignarle profesor de Religión (2 horas)
Al curso 4B no fue posible asignarle profesor de T

In [187]:
model2 = gp.Model("Asignación de cursos a profesores")

prc_para_model2 = trios_prc.copy()
tuplas_placeholder = []
for r,c in duplas_rc:
    tuplas_placeholder.append(('Placeholder', r, c))
tuplas_placeholder

prc_placeholder = prc_para_model2 + tuplas_placeholder

z = model2.addVars(prc_placeholder, vtype=GRB.BINARY, name="z")

In [188]:
model2.addConstrs((sum(z[p,r,c] for p,s,d in prc_placeholder if r == s if c == d) == 1 for r,c in duplas_rc),name="R1");

In [189]:
model2.addConstrs((sum(z[p,r,c] * cantidad_de_horas_por_ramo_por_curso[c][r] for p,r,c in trios_prc if p == q) \
    <= disponibilidad_horaria[q] for q in ramos_y_cursos_dictados_por_profesor),name="R2");

In [190]:
model2.addConstr((sum(z['Placeholder',r,c] * cantidad_de_horas_por_ramo_por_curso[c][r] for r,c in duplas_rc) \
    <= horas_placeholder),name="R3");

In [191]:
model2.addConstrs((z[p,r,c] == 1 for r,p,c in clases_ya_asignadas),name="R5");

In [192]:
obj2 = 3 * sum(z[p,r,c] for p in profesores for r,c in preferencias_profesores[p]) - 2 * sum(z['Placeholder',r,c] for r,c in duplas_rc)
model2.setObjective(obj2, GRB.MAXIMIZE)

model2.write('model.lp')
model2.update()
model2.optimize()
model2.write('out.sol')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 112 rows, 144 columns and 289 nonzeros
Model fingerprint: 0xc9c6df88
Variable types: 0 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -163.0000000
Presolve removed 102 rows and 128 columns
Presolve time: 0.00s
Presolved: 10 rows, 16 columns, 32 nonzeros
Found heuristic solution: objective -123.0000000
Variable types: 0 continuous, 16 integer (16 binary)
Found heuristic solution: objective -121.0000000

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0     

In [193]:
with open ('out.sol') as f:
    ocupaciones_profesores = []
    for line in f.readlines()[2:]:
        if line[-2] == '1' and line[0] == 'z':
            line = line.strip('\n')
            line = line[2:-3]
            line = line.split(',')
            ocupaciones_profesores.append(line)

for elem in ocupaciones_profesores:
    # if elem[0] != 'Placeholder':
    #     if (elem[1], elem[2]) in preferencias_profesores[elem[0]]:
    #         print(elem)
    if elem[0] == 'Placeholder':
        print(elem)

['Placeholder', 'Orientación', '4A']
['Placeholder', 'Educación_Física', '4A']
['Placeholder', 'Lenguaje', '4A']
['Placeholder', 'Ciencias_Naturales', '4A']
['Placeholder', 'Religión', '4A']
['Placeholder', 'Taller_Comprensión_Lectora', '4A']
['Placeholder', 'Música', '4A']
['Placeholder', 'Tecnología', '4A']
['Placeholder', 'Artes_Visuales', '4A']
['Placeholder', 'Orientación', '4B']
['Placeholder', 'Educación_Física', '4B']
['Placeholder', 'Lenguaje', '4B']
['Placeholder', 'Ciencias_Naturales', '4B']
['Placeholder', 'Religión', '4B']
['Placeholder', 'Taller_Comprensión_Lectora', '4B']
['Placeholder', 'Música', '4B']
['Placeholder', 'Tecnología', '4B']
['Placeholder', 'Artes_Visuales', '4B']
['Placeholder', 'Orientación', '5A']
['Placeholder', 'Lenguaje', '5A']
['Placeholder', 'Inglés', '5A']
['Placeholder', 'Ciencias_Naturales', '5A']
['Placeholder', 'Geografía', '5A']
['Placeholder', 'Educación_Física', '5A']
['Placeholder', 'Religión', '5A']
['Placeholder', 'Música', '5A']
['Placeh

In [204]:
def generar_profesores_por_curso(ocupaciones_profesores):
    ocupaciones_profesores = sorted(ocupaciones_profesores, key= lambda x: x[1])
    ocupaciones_profesores = sorted(ocupaciones_profesores, key= lambda x: x[2])

    ultimo_ramo = None
    ultimo_curso = None
    indices = []
    for i in range(len(ocupaciones_profesores)):
        if ocupaciones_profesores[i][1] == ultimo_ramo:
            if ocupaciones_profesores[i][2] == ultimo_curso:
                indices.append(i)
        ultimo_ramo = ocupaciones_profesores[i][1]
        ultimo_curso = ocupaciones_profesores[i][2]
    indices.reverse()

    curso_actual = None
    profesores_por_curso = {}
    for i in range(len(ocupaciones_profesores)):
        if ocupaciones_profesores[i][2] != curso_actual:
            profesores_por_curso[ocupaciones_profesores[i][2]] = {}
            if type(ocupaciones_profesores[i][0]) == str:
                profesores_por_curso[ocupaciones_profesores[i][2]][1] = [[ocupaciones_profesores[i][0], ocupaciones_profesores[i][1]]]
            else:
                profesores_por_curso[ocupaciones_profesores[i][2]][len(ocupaciones_profesores[i][0])] = [[ocupaciones_profesores[i][0], ocupaciones_profesores[i][1]]]
        else:
            if type(ocupaciones_profesores[i][0]) == str:
                if 1 not in profesores_por_curso[ocupaciones_profesores[i][2]]:
                    profesores_por_curso[ocupaciones_profesores[i][2]][1] = [[ocupaciones_profesores[i][0], ocupaciones_profesores[i][1]]]
                else:
                    profesores_por_curso[ocupaciones_profesores[i][2]][1].append([ocupaciones_profesores[i][0], ocupaciones_profesores[i][1]])
            else:
                if len(ocupaciones_profesores[i][0]) not in profesores_por_curso[ocupaciones_profesores[i][2]]:
                    profesores_por_curso[ocupaciones_profesores[i][2]][len(ocupaciones_profesores[i][0])] = [[ocupaciones_profesores[i][0], ocupaciones_profesores[i][1]]]
                else:
                    profesores_por_curso[ocupaciones_profesores[i][2]][len(ocupaciones_profesores[i][0])].append([ocupaciones_profesores[i][0], ocupaciones_profesores[i][1]])
        curso_actual = ocupaciones_profesores[i][2]
    return profesores_por_curso

def generar_carga_profesores(ocupaciones_profesores):
    ocupaciones_profesores = sorted(ocupaciones_profesores, key= lambda x: x[0])
    carga_profesores = {}
    profesor_actual = None
    for i in range(len(ocupaciones_profesores)):
        if ocupaciones_profesores[i][0] != profesor_actual:
            carga_profesores[ocupaciones_profesores[i][0]] = {}
            carga_profesores[ocupaciones_profesores[i][0]]["1"] = [[ocupaciones_profesores[i][2], ocupaciones_profesores[i][1]]]
        else:
            carga_profesores[ocupaciones_profesores[i][0]]["1"].append([ocupaciones_profesores[i][2], ocupaciones_profesores[i][1]])
        profesor_actual = ocupaciones_profesores[i][0]
    return carga_profesores


output = {}
output['profesores_por_curso'] = generar_profesores_por_curso(ocupaciones_profesores)
output['ramos_por_curso'] = cantidad_de_horas_por_ramo_por_curso
output['carga_profesores'] = generar_carga_profesores(ocupaciones_profesores)
with open('probando.json', 'w', encoding='utf8') as f:
    json.dump(output, f, indent=4)